# Transformer model for time series analysis
_Using energy and weather data from Spain_

The first step is to import all the required libraries

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import missingno as mno

from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)


from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel, NaiveSeasonal, NaiveDrift, ExponentialSmoothing, TransformerModel
from darts.utils.statistics import check_seasonality, extract_trend_and_seasonality, plot_acf, plot_hist
from darts.metrics import mape, rmse
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression
from darts.utils.utils import ModelMode, SeasonalityMode, TrendMode

pd.set_option("display.precision",2)
np.set_printoptions(precision=2, suppress=True)
pd.options.display.float_format = '{:,.2f}'.format

# Control settings and constants 
We set a number of hyperparameters to configure the Transformer model and some other constants that represent additional control settings.
Right at the top, note the parameter LOAD.

-When it is set to False, the script will train a Transformer model on the training set.

-When Load is set to True, the script will not retrain the model. Instead, it will load a previously saved Transformer model from the current working directory — the folder in which you have saved the Jupyter notebook. The lengthy training session will be skipped and the script will proceed to generate forecasts from the loaded model.

As soon as a training session completes, the script will automatically save the newly trained model in a tar file. You can modify its file name in line 2.

In [2]:
LOAD = False      # True = load previously saved model from disk?  False = (re)train the model
SAVE = "\_TForm_model10e.pth.tar"   # file name to save the model under

EPOCHS = 200
INLEN = 32          # input size
FEAT = 32           # d_model = number of expected features in the inputs, up to 512    
HEADS = 4           # default 8
ENCODE = 4          # encoder layers
DECODE = 4          # decoder layers
DIM_FF = 128        # dimensions of the feedforward network, default 2048
BATCH = 32          # batch size
ACTF = "relu"       # activation function, relu (default) or gelu
SCHLEARN = None     # a PyTorch learning rate scheduler; None = constant rate
LEARN = 1e-3        # learning rate
VALWAIT = 1         # epochs to wait before evaluating the loss on the test/validation set
DROPOUT = 0.1       # dropout rate
N_FC = 1            # output size

RAND = 42           # random seed
N_SAMPLES = 100     # number of times a prediction is sampled from a probabilistic model
N_JOBS = 3          # parallel processors to use;  -1 = all processors

# default quantiles for QuantileRegression
QUANTILES = [0.01, 0.1, 0.2, 0.5, 0.8, 0.9, 0.99]

SPLIT = 0.9         # train/test %

FIGSIZE = (9, 6)


qL1, qL2 = 0.01, 0.10        # percentiles of predictions: lower bounds
qU1, qU2 = 1-qL1, 1-qL2,     # upper bounds derived from lower bounds
label_q1 = f'{int(qU1 * 100)} / {int(qL1 * 100)} percentile band'
label_q2 = f'{int(qU2 * 100)} / {int(qL2 * 100)} percentile band'

mpath = os.path.abspath(os.getcwd()) + SAVE     # path and file name to save the model

Download the data and save it into the folder https://www.kaggle.com/datasets/nicholasjhana/energy-consumption-generation-prices-and-weather

In [3]:
# load
df0 = pd.read_csv("energy_dataset.csv", header=0, parse_dates=["time"])
dfw0 = pd.read_csv("weather_features.csv", header=0, parse_dates=["dt_iso"])

KeyboardInterrupt: 

# Display the data 

In [ ]:
df0.iloc[[0,-1]]

In [ ]:
dfw0.iloc[[0,-1]]

In [ ]:
# backup of original sources
df1 = df0.copy()
dfw1 = dfw0.copy()

# Lets start with the energy data

In [ ]:
df1.info()

In [ ]:
# datetime
df1["time"] = pd.to_datetime(df1["time"], utc=True, infer_datetime_format=True)


# any duplicate time periods?
print("count of duplicates:",df1.duplicated(subset=["time"], keep="first").sum())


df1.set_index("time", inplace=True)


# any non-numeric types?
print("non-numeric columns:",list(df1.dtypes[df1.dtypes == "object"].index))
# confirms that we do not need to convert any non numerical variable into  numbers


# any missing values?
def gaps(df):
    if df.isnull().values.any():
        print("MISSING values:\n")
        mno.matrix(df)
    else:
        print("no missing values\n")
gaps(df1)  

In [ ]:
# drop the NaN and zero columns, and also the 'forecast' columns
df1 = df1.drop(df1.filter(regex="forecast").columns, axis=1, errors="ignore")
df1.dropna(axis=1, how="all", inplace=True)
df1 = df1.loc[:, (df1!=0).any(axis=0)]


# handle missing values in rows of remaining columns. The next lines susbstitute NaNs with interpolated values.
df1 = df1.interpolate(method ="bfill")
# any missing values left?
gaps(df1)

df1 = df1.loc[:, (df1!=0).any(axis=0)]

In [ ]:
# rename columns
colnames_old = df1.columns
colnames_new = ["gen_bio", "gen_lig", "gen_gas", "gen_coal", \
                "gen_oil", "gen_hyd_pump", "gen_hyd_river", "gen_hyd_res", \
                "gen_nuc", "gen_other", "gen_oth_renew", "gen_solar", \
                "gen_waste", "gen_wind", "load_actual", "price_dayahead", \
                "price"]
dict_cols = dict(zip(colnames_old, colnames_new))
df1.rename(columns=dict_cols, inplace=True)
print(df1.info())
df1.describe()

In [ ]:
# convert int and float64 columns to float32
intcols = list(df1.dtypes[df1.dtypes == np.int64].index)
df1[intcols] = df1[intcols].applymap(np.float32)

f64cols = list(df1.dtypes[df1.dtypes == np.float64].index)
df1[f64cols] = df1[f64cols].applymap(np.float32)

In [ ]:
plt.figure(100, figsize=(20, 7))
sns.lineplot(x = "time", y = "price", data = df1, palette="coolwarm");

# Next, weather data 

In [ ]:
dfw1.info()

We start by converting the “dt_iso” column to datetime format to make it compatible with the datetime index of the energy dataframe.

In [ ]:
# datetime
dfw1["time"] = pd.to_datetime(dfw1["dt_iso"], utc=True, infer_datetime_format=True)
dfw1.set_index("time", inplace=True)


# any non-numeric types?
print("non-numeric columns:",list(dfw1.dtypes[dfw1.dtypes == "object"].index))


# any missing values?
def gaps(df):
    if df.isnull().values.any():
        print("MISSING values:\n")
        mno.matrix(df)
    else:
        print("no missing values\n")  
gaps(dfw1) 


dfw1.describe()


**The descriptive statistics reveal some values that evidently represent outliers.**

The atmospheric pressure cannot rise above a million millibar without crushing even a nuclear submarine.
A wind speed as high as 133 km/h has not been recorded in Spain this century.

The temperature columns look outlandish, with values above 300 degrees. But these values are just expressed in kelvin and therefore don’t suggest obvious outliers.

In [ ]:
# drop unnecessary columns
dfw1.drop(["rain_3h", "weather_id", "weather_main", "weather_description", "weather_icon"], 
          inplace=True, axis=1, errors="ignore")


# temperature: kelvin to celsius
temp_cols = [col for col in dfw1.columns if "temp" in col]
dfw1[temp_cols] = dfw1[temp_cols].filter(like="temp").applymap(lambda t: t - 273.15)

# convert int and float64 columns to float32
intcols = list(dfw1.dtypes[dfw1.dtypes == np.int64].index)
dfw1[intcols] = dfw1[intcols].applymap(np.float32)

f64cols = list(dfw1.dtypes[dfw1.dtypes == np.float64].index)
dfw1[f64cols] = dfw1[f64cols].applymap(np.float32)

f32cols = list(dfw1.dtypes[dfw1.dtypes == np.float32].index)
dfw1.info()

Next, we need to investigate the outliers which we saw in the descriptive statistics table.

The atmospheric pressure is implausibly high on a few days in February 2015. Their normal range is in a narrow band around 1000 millibar.

In [ ]:
#investigate the outliers in the pressure column
dfw1["pressure"].nlargest(10)

In [ ]:
#investigate the outliers in the wind_speed column
dfw1["wind_speed"].nlargest(10)

**Boxplots can help us to obtain visual clues on outliers.**

In [ ]:
# boxplots
for i, c in enumerate(f32cols):
    sns.boxplot(x=dfw1[c], palette="coolwarm")
    plt.show();

Alternatively, we can use seaborn’s distplots to identify outliers.

The temperature curve shows a normal seasonal cycle, from freezing winter cold to seething summer heat. Without a comparison with third-party data, we won’t discern obvious temperate outliers.

But the extreme kurtosis and the skew of the pressure and wind speed distributions reveal the existence of outliers in their right tails.

In [ ]:
# or use distplot to visualize outliers
fig = plt.figure(figsize=(5, 4)) 
ax = sns.distplot(dfw1["temp"])
xmin = dfw1["temp"].min()
xmax = dfw1["temp"].max()  
ax.set_xlim(xmin, xmax)
ax.set_title("temp");

fig = plt.figure(figsize=(5, 4))
ax = sns.distplot(dfw1["pressure"])
xmin = dfw1["pressure"].min()
xmax = dfw1["pressure"].max()  
ax.set_xlim(xmin, xmax)
ax.set_title("pressure");

fig = plt.figure(figsize=(5, 4))
ax = sns.distplot(dfw1["wind_speed"])
xmin = dfw1["wind_speed"].min()
xmax = dfw1["wind_speed"].max()  
ax.set_xlim(xmin, xmax)
ax.set_title("wind_speed");

In [ ]:
# treatment of outliers: replace with NaN, then interpolate
dfw1["pressure"].where( dfw1["pressure"] <= 1050, inplace=True)
dfw1["pressure"].where( dfw1["pressure"] >= 948, inplace=True)
dfw1["wind_speed"].where( dfw1["wind_speed"] <= 120, inplace=True)
dfw1["clouds_all"].where( dfw1["clouds_all"] <= 40, inplace=True)
dfw1 = dfw1.interpolate(method ="bfill")

sns.boxplot(x=dfw1["pressure"], palette="coolwarm")
plt.show();
sns.boxplot(x=dfw1["wind_speed"], palette="coolwarm")
plt.show();
sns.boxplot(x=dfw1["clouds_all"], palette="coolwarm")
plt.show();

dfw1.describe()

Now, lets make sure that both data sets have the same time index

In [ ]:
# start and end of energy and weather time series 
print("earliest weather time period:", dfw1.index.min())
print("latest weather time period:", dfw1.index.max())

print("earliest energy time period:", df1.index.min())
print("latest energy time period:", df1.index.max())

In [ ]:
# cities in weather data
cities = dfw1["city_name"].unique()
cities

In [ ]:
# drop duplicate time periods
print("count of duplicates before treatment:",dfw1.duplicated(subset=["dt_iso", "city_name"], keep="first").sum())

dfw1 = dfw1.drop_duplicates(subset=["dt_iso", "city_name"], keep="first")
dfw1.reset_index()
print("count of duplicates after treatment:",dfw1.duplicated(subset=["dt_iso", "city_name"], keep="first").sum())

# set datetime index
dfw1["time"] = pd.to_datetime(dfw1["dt_iso"], utc=True, infer_datetime_format=True)
dfw1.set_index("time", inplace=True)
dfw1.drop("dt_iso", inplace=True, axis=1)


print("size of energy dataframe:", df1.shape[0])
dfw1_city = dfw1.groupby("city_name").count()
dfw1_city



We isolate each city’s weather records in a dataframe of its own to prepare for the merger of the energy and weather records. The dictionary comprehension in line 2 creates the city-specific dataframes.

The city names serve as the dictionary keys. When we select, for instance, the key “Bilbao”, we can retrieve the Basque city’s dataframe from the dictionary.



In [ ]:
# count of weather observations by city
print("size of energy dataframe:", df1.shape[0])

dfw1["city_name"] = dfw1["city_name"].replace(" Barcelona", "Barcelona")   # remove space in name
dfw1_city = dfw1.groupby("city_name")
print("size of city groups in weather dataframe:")
dfw1_city.count()

In [ ]:
# separate the cities: a weather dataframe for each of them
# One must redefine dfw1_city in order to remove the .count() command. 

dict_city_weather = {city:df_city for city,df_city in dfw1_city}
dict_city_weather.keys()

**Two examples: Barcelona and Bilbao**

_Note that " Barcelona" has a space at the begining of the word_

In [ ]:
# example: Bilbao weather dataframe
dfw_Bilbao = dict_city_weather.get("Bilbao")
print("Bilbao weather:")
dfw_Bilbao.describe()


In [ ]:
# example: Barcelona weather dataframe
dfw_Barcelona = dict_city_weather.get("Barcelona")
print("Barcelona weather:")
dfw_Barcelona.describe()


# Merger of the Energy and Weather Data


In [ ]:
# merge the energy and weather dataframes
df2 = df1.copy()
for city,df in dict_city_weather.items():
    city_name = str(city) + "_"
    df = df.add_suffix("_{}".format(city))
    df2 = pd.concat([df2, df], axis=1)
    df2.drop("city_name_" + city, inplace=True, axis=1)
print(df2.info())
df2.iloc[[0,-1]]

In [ ]:
column_headers = list(df2.columns.values)
print("The Column Header :", column_headers)

In [ ]:
# any null values?
print("any missing values?", df2.isnull().values.any())

# any ducplicate time periods?
print("count of duplicates:", df2.duplicated(keep="first").sum())

In an out-of-sample forecast, we want to estimate the electricity prices over the next few hours. It is doubtful that historical prices, or any of the feature variables, have formed patterns that have persisted over several years and will influence the prices we will observe 12 hours from now. I choose to limit the source data to the 8,760 hours of the final year, January to December 2018. Otherwise, the training time would have been multiplied. The training would take half a day or night.

In [ ]:
# limit the dataframe's date range
df2 = df2[df2.index >= "2018-01-01 00:00:00+00:00"]
df2.iloc[[0,-1]]

In [ ]:
# check correlations of features with price
df_corr = df2.corr(method="pearson")
print(df_corr.shape)
print("correlation with price:")
df_corrP = pd.DataFrame(df_corr["price"].sort_values(ascending=False))
df_corrP

In [ ]:
# highest absolute correlations with price
pd.options.display.float_format = '{:,.2f}'.format
df_corrH = df_corrP[np.abs(df_corrP["price"]) > 0.3]
df_corrH

In [ ]:
# helper method: correlation matrix as heatmap
def corr_heatmap(df):
    idx = df.corr().sort_values("price", ascending=False).index
    df_sorted = df.loc[:, idx]  # sort dataframe columns by their correlation 

    #plt.figure(figsize = (15,15))
    sns.set(font_scale=0.75)
    ax = sns.heatmap(df_sorted.corr().round(3), 
            annot=True, 
            square=True, 
            linewidths=.75, cmap="coolwarm", 
            fmt = ".2f", 
            annot_kws = {"size": 11})
    ax.xaxis.tick_bottom()
    plt.title("correlation matrix")
    plt.show()

In [ ]:
# call helper function to plot correlation heatmap
df3 = df2.copy()
df3 = df3[df_corrH.index]
plt.figure(figsize = (15,15))
corr_heatmap(df3)

We can choose between two approaches.

The first one proceeds with those 28 original features which are at least moderately correlated with the prices. The alternative would be a principal component analysis that reduces the 66 features to, hopefully, just a handful of their linear combinations.

Lets behind with the principal component solutions (PCA). 

In [ ]:
# PCA:
# dataframe with feature columns only, without actual price
df3 = df2.copy()
df_feat = df3.loc[:, df3.columns != "price"]
#print(df_feat.info())
df_feat = MinMaxScaler().fit_transform(df_feat)

# principal components among features
pca = PCA(n_components=30)
res_pca = pca.fit_transform(df_feat)

# scree plot
features = range(pca.n_components_)
plt.figure(figsize = (25,10))
plt.bar(features, pca.explained_variance_ratio_, color="blue")
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("PCA features", fontsize=15)
plt.ylabel("% variance explained", fontsize=20)
plt.xticks(features, fontsize=20)
plt.yticks(ticks=np.arange(0.0, 1.0001, 0.1), fontsize=20);

The principal component analysis demonstrates that as many as 30 components would be needed to reach a cumulative variance ratio higher than 90%. Thus, PCA does not condense the feature columns as much as we would wish.

In [ ]:
# collect principal components in a dataframe
df_pca = pd.DataFrame(res_pca)
df_pca = df_pca.add_prefix("pca")


df3 = df2.copy()
col = df3.pop("price")
df3.insert(0, col.name, col)     # move price column to the left

# drop the feature columns which the components summarize
df3.reset_index(inplace=True)
df3 = pd.concat([df3, df_pca], axis=1)
for col in df3.columns:
    if col != "price" and col != "time" and "pca" not in col:
        del df3[col]
df3.set_index("time", inplace=True)
df3.columns

Let’s compute their correlations with the price level and limit the output to those components which have at least 10% correlation with it.

In [ ]:
# prepare correlation matrix for principal components  
df_corr = df3.corr(method="pearson")
print("principal components with at least modest correlation to price:")
df_corrP = pd.DataFrame(df_corr["price"].sort_values(ascending=False))

# absolute correlations with price > 10%
pd.options.display.float_format = '{:,.2f}'.format
df_corrH = df_corrP[np.abs(df_corrP["price"]) > 0.10]
df_corrH

Lets define a helper function to make things easier. 

In [ ]:
# visualize correlations with price     
df4 = df3.copy()
df4 = df4[df_corrH.index]   # keep the components with at least modest correlations

plt.figure(figsize = (10,10))
corr_heatmap(df4)

In [ ]:
# additional datetime columns to discern patterns along time axis
df4["month"] = df4.index.month

df4["wday"] = df4.index.dayofweek
dict_days = {0:"1_Mon", 1:"2_Tue", 2:"3_Wed", 3:"4_Thu", 4:"5_Fri", 5:"6_Sat", 6:"7_Sun"}
df4["weekday"] = df4["wday"].apply(lambda x: dict_days[x])

df4["hour"] = df4.index.hour

df4 = df4.astype({"hour":float, "wday":float, "month": float})

df4.iloc[[0, -1]]

In [ ]:
# pivot table: weekdays in months
piv = pd.pivot_table(   df4, 
                        values="price", 
                        index="month", 
                        columns="weekday", 
                        aggfunc="mean", 
                        margins=True, margins_name="Avg", 
                        fill_value=0)
pd.options.display.float_format = '{:,.0f}'.format

plt.figure(figsize = (7,15))
sns.set(font_scale=1)
sns.heatmap(piv.round(0), annot=True, square = True, \
            linewidths=.75, cmap="coolwarm", fmt = ".0f", annot_kws = {"size": 12})
plt.title("price by weekday by month")
plt.show()

In [ ]:
# pivot table: hours in weekdays
piv = pd.pivot_table(   df4, 
                        values="price", 
                        index="hour", 
                        columns="weekday", 
                        aggfunc="mean", 
                        margins=True, margins_name="Avg", 
                        fill_value=0)
pd.options.display.float_format = '{:,.0f}'.format

plt.figure(figsize = (7,20))
sns.set(font_scale=1)
sns.heatmap(piv.round(0), annot=True, square = True, \
            linewidths=.75, cmap="coolwarm", fmt = ".0f", annot_kws = {"size": 12})
plt.title("price by hour by weekday")
plt.show()

In [ ]:
# dataframe with price and feature columns only
df4.drop(["weekday", "month", "wday", "hour"], inplace=True, axis=1)
# print(df4.info())

# Time series object

In [ ]:
# create time series object for the target variable
ts_P = TimeSeries.from_series(df4["price"]) 

# check attributes of the time series
print("components:", ts_P.components)
print("duration:",ts_P.duration)
print("frequency:",ts_P.freq)
print("frequency:",ts_P.freq_str)
print("has date time index? (or else, it must have an integer index):",ts_P.has_datetime_index)
print("deterministic:",ts_P.is_deterministic)
print("univariate:",ts_P.is_univariate)

In [ ]:
# create a time series object for the feature columns
df_covF = df4.loc[:, df4.columns != "price"]
ts_covF = TimeSeries.from_dataframe(df_covF)


# check attributes of the time series
print("components (columns) of feature time series:", ts_covF.components)
print("duration:",ts_covF.duration)
print("frequency:",ts_covF.freq)
print("frequency:",ts_covF.freq_str)
print("has date time index? (or else, it must have an integer index):",ts_covF.has_datetime_index)
print("deterministic:",ts_covF.is_deterministic)
print("univariate:",ts_covF.is_univariate)

In [ ]:
# example: operating with time series objects:
# we can also create a 3-dimensional numpy array from a time series object
# 3 dimensions: time (rows) / components (columns) / samples
ar_covF = ts_covF.all_values()
print(type(ar_covF))
ar_covF.shape

In [ ]:
# example: operating with time series objects:
# we can also create a pandas series or dataframe from a time series object
df_covF = ts_covF.pd_dataframe()
type(df_covF)

In [ ]:
# train/test split and scaling of target variable
ts_train, ts_test = ts_P.split_after(SPLIT)
print("training start:", ts_train.start_time())
print("training end:", ts_train.end_time())
print("training duration:",ts_train.duration)
print("test start:", ts_test.start_time())
print("test end:", ts_test.end_time())
print("test duration:", ts_test.duration)


scalerP = Scaler()
scalerP.fit_transform(ts_train)
ts_ttrain = scalerP.transform(ts_train)
ts_ttest = scalerP.transform(ts_test)    
ts_t = scalerP.transform(ts_P)

# make sure data are of type float
ts_t = ts_t.astype(np.float32)
ts_ttrain = ts_ttrain.astype(np.float32)
ts_ttest = ts_ttest.astype(np.float32)

print("first and last row of scaled price time series:")
pd.options.display.float_format = '{:,.2f}'.format
ts_t.pd_dataframe().iloc[[0,-1]]


In [ ]:
# train/test split and scaling of feature covariates
covF_train, covF_test = ts_covF.split_after(SPLIT)

scalerF = Scaler()
scalerF.fit_transform(covF_train)
covF_ttrain = scalerF.transform(covF_train) 
covF_ttest = scalerF.transform(covF_test)   
covF_t = scalerF.transform(ts_covF)  

# make sure data are of type float
covF_ttrain = ts_ttrain.astype(np.float32)
covF_ttest = ts_ttest.astype(np.float32)

In [ ]:
# feature engineering - create time covariates: hour, weekday, month, year, country-specific holidays
covT = datetime_attribute_timeseries(ts_P.time_index, attribute="hour", one_hot=False)
covT = covT.stack(datetime_attribute_timeseries(ts_P.time_index, attribute="day_of_week", one_hot=False))
covT = covT.stack(datetime_attribute_timeseries(ts_P.time_index, attribute="month", one_hot=False))
covT = covT.stack(datetime_attribute_timeseries(ts_P.time_index, attribute="year", one_hot=False))

covT = covT.add_holidays(country_code="ES")
covT = covT.astype(np.float32)


# train/test split
covT_train, covT_test = covT.split_after(SPLIT)


# rescale the covariates: fitting on the training set
scalerT = Scaler()
scalerT.fit(covT_train)
covT_ttrain = scalerT.transform(covT_train)
covT_ttest = scalerT.transform(covT_test)
covT_t = scalerT.transform(covT)
covT_t = covT_t.astype(np.float32)


pd.options.display.float_format = '{:.0f}'.format
print("first and last row of unscaled time covariates:")
covT.pd_dataframe().iloc[[0,-1]]

In [ ]:
# combine feature covariates and time covariates in a single time series object
ts_cov = ts_covF.concatenate(covT, axis=1)                      # unscaled F+T
cov_t = covF_t.concatenate(covT_t, axis=1)                      # scaled F+T
cov_ttrain = covF_ttrain.concatenate(covT_ttrain, axis=1)       # scaled F+T training

print("first and last row of unscaled covariates:")
ts_cov.pd_dataframe().iloc[[0,-1]]

# Training the transformer

In [ ]:
model = TransformerModel(
                    input_chunk_length = INLEN,
                    output_chunk_length = N_FC,
                    batch_size = BATCH,
                    n_epochs = EPOCHS,
                    model_name = "Transformer_price",
                    nr_epochs_val_period = VALWAIT,
                    d_model = FEAT,
                    nhead = HEADS,
                    num_encoder_layers = ENCODE,
                    num_decoder_layers = DECODE,
                    dim_feedforward = DIM_FF,
                    dropout = DROPOUT,
                    activation = ACTF,
                    random_state=RAND,
                    likelihood=QuantileRegression(quantiles=QUANTILES), 
                    optimizer_kwargs={'lr': LEARN},
                    add_encoders={"cyclic": {"future": ["hour", "dayofweek", "month"]}},
                    save_checkpoints=True,
                    force_reset=True
                    )

In [ ]:

# training: load a saved model or (re)train
if LOAD:
    print("have loaded a previously saved model from disk:" + mpath)
    model = TransformerModel.load_model(mpath)                            # load previously model from disk 
else:
    model.fit(  ts_ttrain, 
                past_covariates=cov_t, 
                verbose=True)
    print("have saved the model after training:", mpath)
    model.save_model(mpath)

In [ ]:
# save model to current working directory
print("saved model:", mpath)
model.save_model(mpath)

In [ ]:
# testing: generate predictions
ts_tpred = model.predict(n=len(ts_ttest),
                         num_samples=N_SAMPLES,
                            n_jobs=N_JOBS)

In [ ]:
# retrieve forecast series for chosen quantiles, 
# inverse-transform each series,
# insert them as columns in a new dataframe dfY
q50_RMSE = np.inf
q50_MAPE = np.inf
ts_q50 = None
pd.options.display.float_format = '{:,.2f}'.format
dfY = pd.DataFrame()
dfY["Actual"] = TimeSeries.pd_series(ts_test)


# helper function: get forecast values for selected quantile q and insert them in dataframe dfY
def predQ(ts_t, q):
    ts_tq = ts_t.quantile_timeseries(q)
    ts_q = scalerP.inverse_transform(ts_tq)
    s = TimeSeries.pd_series(ts_q)
    header = "Q" + format(int(q*100), "02d")
    dfY[header] = s
    if q==0.5:
        ts_q50 = ts_q
        q50_RMSE = rmse(ts_q50, ts_test)
        q50_MAPE = mape(ts_q50, ts_test) 
        print("RMSE:", f'{q50_RMSE:.2f}')
        print("MAPE:", f'{q50_MAPE:.2f}')
  
    
# call helper function predQ, once for every quantile
_ = [predQ(ts_tpred, q) for q in QUANTILES]

# move Q50 column to the left of the Actual column
col = dfY.pop("Q50")
dfY.insert(1, col.name, col)
dfY.iloc[np.r_[0:2, -2:0]]

In [ ]:
plt.figure(100, figsize=(20, 7))
sns.set(font_scale=1.3)
p = sns.lineplot(x="time", y="Q50", data=dfY, palette="coolwarm")
sns.lineplot(x="time", y="Actual", data=dfY, palette="coolwarm")
plt.legend(labels=["forecast median price Q50", "actual price"])
p.set_ylabel("price")
p.set_xlabel("")
p.set_title("energy price");

In [ ]:

# choose forecast horizon: k hours beyond end of test set
k = 12   

n_FC = k + len(ts_ttest)   # length of test set + k hours
print("forecast beyond end of training set:", n_FC, 
      "hours beyond", ts_ttrain.end_time())

# last 24 hours of feature covariates available => copy them to future 24 hours:
covF_t_fut = covF_t.concatenate(other=covF_t.tail(size=24), 
                                    ignore_time_axes=True)
covF_t_fut.pd_dataframe()

In [ ]:
covT_t.pd_dataframe()

In [ ]:
# combine feature and time covariates:
# cov_t_fut = covF_t_fut.concatenate(covT_t.slice_intersect(covF_t_fut), axis=1) 
# cov_t_fut.pd_dataframe().iloc[[0,-1]]





###### Im having trouble putting together here. Lets try without PCA first to see. 